In [3]:
!pip install langchain langchain-groq


In [4]:
from langchain_groq import ChatGroq

In [7]:
# Initialize LangChain ChatGroq model with API key
llm = ChatGroq(
    groq_api_key="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",
    model_name="gemma2-9b-it",  # or "mixtral-8x7b" / "llama3-8b-8192" / etc.
    temperature=0.7
)

In [8]:
ans=llm.invoke("What is the national sport of India?")

In [9]:
print(ans.content)

The national sport of India is **hockey**. 



In [10]:
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 810.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 41.6 MB/s eta 0:00:00


In [11]:
!pip install langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [12]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

# Load Embedding Model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


/tmp/ipython-input-13-4275027608.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warning

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/content/Sayandeep Resume.pdf')

In [16]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [17]:
raw_text

'Sayandeep Sarkar +91- 7001932512\nBachelor of Engineering sarkarsayandeep093@gmail.com\nin Mechanical Engineering GitHub\nJadavpur University, Kolkata, India LinkedIn\nSummary\nRecent graduate from Jadavpur University with a strong interest in Machine Learning, Data Science, and Software\nDevelopment.\nSPECIALIST - Codeforces (1400*) (solved 350+ questions)\nSolved 550+ Data Structures and Algorithms questions on Leetcode and overall 1000+ questions including Codeforces\nand Gfg\nInternship\nData Engineering Intern at IIM Shillong\nDeveloped and managed ETL pipelines for large-scale data processing. Workflows built and monitored with Apache\nAirflow. Used FastAPI to build data services, Docker for containerization,and MLflow to track ML experiments.\nIntegrated CI pipelines for streamlined development.\nRoles and Responsibilities\nCentral Placement Coordinator of Jadavpur University for 2025 batch\nEducation\nDegree Institute Board / University CGPA/Percentage Year\nBE Mechanical Engi

In [18]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [19]:
len(texts)

6

In [20]:
from langchain.vectorstores import FAISS

document_search = FAISS.from_texts(texts, embedding=embeddings)

In [21]:
document_search

In [22]:
from langchain.chains.question_answering import load_qa_chain

# Load the QA chain
chain = load_qa_chain(llm, chain_type="stuff")


/tmp/ipython-input-22-3394503864.py:4: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


In [23]:
query = "Who is Sayandeep Sarkar?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

/tmp/ipython-input-23-3114807077.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=docs, question=query)


'Based on the provided text, Sayandeep Sarkar is a recent graduate from Jadavpur University with a Bachelor of Engineering in Mechanical Engineering. \n\nHe has a strong interest in Machine Learning, Data Science, and Software Development and has considerable experience solving algorithmic problems on platforms like Codeforces, Leetcode, and GFG. \n\nSayandeep has also completed an internship as a Data Engineering Intern at IIM Shillong, where he worked with tools like Apache Airflow, FastAPI, Docker, and MLflow.\n\nHe has developed several projects, including an intelligent web application called "Epiphany" that uses Google Generative AI and FAISS for advanced search capabilities, and a model for predicting concrete compressive strength using various machine learning algorithms.  \n\nYou can find his GitHub profile and LinkedIn profile in the text. \n'

In [24]:
query = "what are the skills he has?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

'Based on the provided text, here are Sayandeep Sarkar\'s skills:\n\n**Technical Skills:**\n\n* **Machine Learning:**  He has experience with machine learning algorithms and has built a project predicting concrete compressive strength using machine learning.\n* **Deep Learning:**  While not explicitly stated, his interest in Machine Learning likely extends to Deep Learning.\n* **GenAI:** He has used Google Generative AI in his "Epiphany" project.\n* **SQL:**  Listed under "Actions" suggesting familiarity.\n* **DBMS:** Listed under "Actions" suggesting familiarity.\n* **OOPS:** Listed under "Actions" suggesting familiarity.\n* **Operating System:** Listed under "Actions" suggesting familiarity.\n* **Data Warehousing:** Listed under "Actions" suggesting familiarity.\n* **Computer Networks:** Listed under "Actions" suggesting familiarity.\n* **Python:**  His primary programming language, used extensively in projects and internships.\n* **Flask:**  A web framework he has used.\n* **FastAPI

# **Evaluate RAG using RAGAS**

In [25]:
!pip install ragas datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.2 MB/s eta 0:00:00


In [40]:
# Define queries and expected responses manually
sample_queries = [
    "Who is Sayandeep Sarkar?",
    "What is Sayandeep's educational background?",
    "Which skills does Sayandeep have?"
]

expected_responses = [
    "Sayandeep Sarkar is the person described in the resume.",
    "Sayandeep completed his education at Jadavpur University.",
    "Sayandeep has skills in data engineering, data science, and software development."
]

In [41]:
# Construct dataset for RAGAS
dataset = []

for query, reference in zip(sample_queries, expected_responses):
    retrieved_docs = document_search.similarity_search(query)
    response = chain.run(input_documents=retrieved_docs, question=query)
    dataset.append({
        "user_input": query,
        "retrieved_contexts": [doc.page_content for doc in retrieved_docs],
        "response": response,
        "reference": reference
    })

In [42]:
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

In [43]:
# Prepare RAGAS Dataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [45]:
# Wrap your LLM
evaluator_llm = LangchainLLMWrapper(llm)

In [46]:
# Evaluate using selected metrics
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[
        LLMContextRecall(),
        Faithfulness(),
        FactualCorrectness()
    ],
    llm=evaluator_llm
)


Evaluating:   0%|          | 0/9 [00:00<?, ?it/s]

In [47]:
print(result)

{'context_recall': 1.0000, 'faithfulness': 0.9889, 'factual_correctness(mode=f1)': 0.0000}


**context_recall:** 1.0000 – The retrieved documents contained all the information needed to answer the question.

**faithfulness:** 0.9889 – The model's answer was almost entirely consistent with the retrieved context.

**factual_correctness(mode=f1):** 0.0000 – The model's answer did not match the expected reference answer in wording or factual content.

In [48]:
for sample in dataset:
    print("Query:", sample["user_input"])
    print("Generated:", sample["response"])
    print("Reference:", sample["reference"])
    print("-" * 50)


Query: Who is Sayandeep Sarkar?
Generated: Sayandeep Sarkar is a recent graduate of Jadavpur University with a Bachelor of Engineering in Mechanical Engineering. He has a strong interest in Machine Learning, Data Science, and Software Development. 

Reference: Sayandeep Sarkar is the person described in the resume.
--------------------------------------------------
Query: What is Sayandeep's educational background?
Generated: Here is Sayandeep's educational background:

* **BE Mechanical Engineering** - Jadavpur University, Kolkata - 7.2 CGPA (2025)
* **Senior Secondary** - Falakata High School - 95.40% (2020) 
* **Matriculation** - Falakata High School - 95.14% (2018)  

Reference: Sayandeep completed his education at Jadavpur University.
--------------------------------------------------
Query: Which skills does Sayandeep have?
Generated: Here are the skills Sayandeep has based on the provided context:

**Programming Languages**
* C/C++
* Python
* Javascript

**Tools and Frameworks**